In [1]:
import os
import requests
import numpy as np
import pandas as pd
from PIL import Image
from io import BytesIO
import psycopg2
import psycopg2.extras as extras

In [41]:
# установка заголовка ключа 
HEAD = 'X-API-KEY'
# установка пути хранения файлов картинок
PATH_IMAGE = './image'

In [42]:
# чтение api ключа
with open('api_key.txt', 'r') as file:
    api_key = file.read()

In [43]:
# чтение данных с сайта минкультуры
url = 'https://opendata.mkrf.ru/v2/cinema/$?l=100'
headers = {HEAD : api_key}
responce = requests.get(url, headers=headers)
json_respons = responce.json()

In [44]:
json_respons.keys()

dict_keys(['status', 'nextPage', 'cursor', 'total', 'filter', 'o', 's', 'l', 'data', 'count'])

In [45]:
json_respons['data']

[{'_id': '5da83099bd36ae31050dd282',
  'hash': '2019-08-19T08:25:21Z',
  'nativeId': '39672',
  'activated': '2019-10-17T09:12:57.336Z',
  'data': {'info': {'path': '/cdm/v2/places/39672',
    'category': 'place',
    'createDate': '2019-08-13T07:02:31Z',
    'updateDate': '2019-08-19T08:25:21Z'},
   'general': {'id': 39672,
    'name': 'Частные кинозалы Black Room',
    'description': '<p>Индивидуальные кинозалы, в которых гости смогут посмотреть любимые фильмы, попеть караоке, поиграть в настольные или видеоигры или просто устроить романтическое свидание. Здесь также можно отметить день рождения, девичник или любой праздник в незаурядной обстановке. </p><p>В распоряжении посетителей – индивидуальные кинозалы, уютные диваны и удобные пуфы, большие экраны, современная аудио– и видеоаппаратура, игровые приставки с шлемом виртуальной реальности и беспроводным сенсором Kinect, благодаря которым можно устраивать соревнования или дэнс-батлы всей компанией, а также караоке и настольные игры.

In [46]:
len(json_respons['data'])

100

In [47]:
for i in json_respons['data'][0]['data']['general'].keys():
    print(i,': ', json_respons['data'][0]['data']['general'][i])
    print()

id :  39672

name :  Частные кинозалы Black Room

description :  <p>Индивидуальные кинозалы, в которых гости смогут посмотреть любимые фильмы, попеть караоке, поиграть в настольные или видеоигры или просто устроить романтическое свидание. Здесь также можно отметить день рождения, девичник или любой праздник в незаурядной обстановке. </p><p>В распоряжении посетителей – индивидуальные кинозалы, уютные диваны и удобные пуфы, большие экраны, современная аудио– и видеоаппаратура, игровые приставки с шлемом виртуальной реальности и беспроводным сенсором Kinect, благодаря которым можно устраивать соревнования или дэнс-батлы всей компанией, а также караоке и настольные игры.</p>

status :  accepted

address :  {'street': 'пр-кт Чайковского,д 84А', 'comment': '', 'fiasHouseId': '39de1e51-4b7b-43f2-842e-cb70fb5ebf12', 'fiasStreetId': '7ccfa312-f149-46ed-a2cb-bed0cebaacb4', 'fiasCityId': 'c52ea942-555e-45c6-9751-58897717b02f', 'fiasRegionId': '61723327-1c20-42fe-8dfa-402638d9b396', 'fullAddress':

In [48]:
for i in json_respons['data'][1]['data']['general'].keys():
    print(i,': ', json_respons['data'][1]['data']['general'][i])
    print()

id :  39429

name :  Кинотеатр «Горизонт. Cinema&Emotion»

description :  <p>13-зальный мультиплекс «Горизонт. Cinema&amp;Emotion» рассчитан на 1 580 мест. </p><p>Абсолютно каждый зал отличается друг от друга концепцией, дизайном и той идеей, которая воплощается в элементах интерьера.</p><p>Кинотеатр – это:</p><ul><li>звук Dolby Atmos (США);</li><li>лазерные системы RGB (Италия);</li><li>проекторы Barco 4K (Бельгия);</li><li>кресла Ferco (Малайзия);</li><li>увеличенное расстояние между рядами кресел;</li><li>7 VIP-залов. </li></ul><p>В 13-зальном кинотеатре выделена VIP-зона из 7 залов, оборудованных по самым высоким международным стандартам. По уровню комфортаVIP-зоны ориентированы на бизнес-классы лучших международных авиалиний. Все залы максимально удобны, с увеличенной шириной проходов между рядами кресел. А кресла для каждого зала приобретены у одного из лучших мировых производителей. </p><p>Театральный зал. В этом зале будет транслироваться альтернативный контент, который не любя

создание таблицы image

In [60]:
# создаем данные для столбцов
image_url = []
image_name = []
image_organization_name = []
for i in range(len(json_respons['data'])):
    if 'gallery' in json_respons['data'][i]['data']['general']:
        list_image_url = [x['url'] for x in json_respons['data'][i]['data']['general']['gallery']]
        image_url.extend(list_image_url)
        image_name.extend([x['title'] for x in json_respons['data'][i]['data']['general']['gallery']])
        for j in range(len(list_image_url)):
            image_organization_name.append(json_respons['data'][i]['data']['general']['organization']['name'])
    if 'image' in json_respons['data'][i]['data']['general']:
        image_url.append(json_respons['data'][i]['data']['general']['image']['url'])
        image_name.append(json_respons['data'][i]['data']['general']['image']['title'])
        image_organization_name.append(json_respons['data'][i]['data']['general']['organization']['name'])

In [ ]:
# загрузка картинок на диск
for i in range(len(image_name)):
    responce_image = requests.get(image_url[i])
    img = Image.open(BytesIO(responce_image.content))
    img.save(PATH_IMAGE + '/'+str(i) + '__' + image_name[i])

In [50]:
image_path = os.listdir(PATH_IMAGE)

In [61]:
# создание таблицы
image_table = pd.DataFrame()
image_table['id'] = range(1, len(image_url)+1)
image_table['url'] = image_url
image_table['name'] = image_name
image_table['organization'] = image_organization_name
image_table['description'] = 'description'
image_table['path'] = np.nan
for i in range(len(image_name)):
    for j in range(len(image_path)):
        if image_name[i] in image_path[j]:
            image_table.loc[i, 'path'] = PATH_IMAGE + '/' + image_path[j]
            break

In [62]:
image_table = image_table[['id', 'url', 'path', 'name', 'organization', 'description']]
image_table

,id,url,path,name,organization,description
0,1,https://all.culture.ru/uploads/3d364e8301aa3c4...,./image/0__XXL (14).jpg,XXL (14).jpg,Частный кинотеатр Black room,description
1,2,https://all.culture.ru/uploads/2501b3dd0ae655d...,./image/1__XXL (5).jpg,XXL (5).jpg,Частный кинотеатр Black room,description
2,3,https://all.culture.ru/uploads/5f76a3b2b98b654...,./image/2__XXL (4).jpg,XXL (4).jpg,Частный кинотеатр Black room,description
3,4,https://all.culture.ru/uploads/bd266dd3b09e68e...,./image/3__XXL (10).jpg,XXL (10).jpg,Частный кинотеатр Black room,description
4,5,https://all.culture.ru/uploads/82fcce8bdaea4a3...,./image/4__XXL (9).jpg,XXL (9).jpg,Частный кинотеатр Black room,description
...,...,...,...,...,...,...
250,251,https://all.culture.ru/uploads/4ab4cb7bd93668d...,./image/250__0000000003.jpg,0000000003.jpg,Сеть кинотеатров «КАРО»,description
251,252,https://all.culture.ru/uploads/502634891c8d94f...,./image/251__IMG_0168.jpg,IMG_0168.jpg,Сеть кинотеатров «КАРО»,description
252,253,https://all.culture.ru/uploads/ccb614f15a686e1...,./image/252__img_8958-2 (1).jpg,img_8958-2 (1).jpg,Сеть кинотеатров «КАРО»,description
253,254,https://all.culture.ru/uploads/dbd40a06c002480...,./image/253__4 151027_LIN_8549.jpg,4 151027_LIN_8549.jpg,Сеть кинотеатров «КАРО»,description


создание таблицы event_image

In [14]:
event_image_table = pd.DataFrame()
event_image_table['id_image'] = image_table['id']
event_image_table['id'] = range(1,len(event_image_table['id_image'])+1)
event_image_table['likes'] = np.random.randint(100, size=len(event_image_table['id_image']))
event_image_table['shows'] = event_image_table['likes'] + np.random.randint(100, size=len(event_image_table['likes']))

In [15]:
event_image_table = event_image_table[['id_image', 'id', 'likes', 'shows']]
event_image_table

,id_image,id,likes,shows
0,1,1,23,89
1,2,2,10,86
2,3,3,97,175
3,4,4,82,127
4,5,5,17,69
...,...,...,...,...
250,251,251,32,101
251,252,252,32,50
252,253,253,59,144
253,254,254,52,149


создание таблицы organization

In [16]:
organization_table = pd.DataFrame()
for i in range(len(json_respons['data'])):
    organization_table.loc[i, 'id'] = int(i) + 1
    organization_table.loc[i, 'name'] = json_respons['data'][i]['data']['general']['organization']['name']
    organization_table.loc[i, 'address'] = json_respons['data'][i]['data']['general']['organization']['address']['fullAddress']
    if 'inn' in json_respons['data'][i]['data']['general']['organization']:
        organization_table.loc[i, 'inn'] = json_respons['data'][i]['data']['general']['organization']['inn']
    else:
        organization_table.loc[i, 'inn'] = np.nan
    organization_table.loc[i, 'description_org'] = json_respons['data'][i]['data']['general']['description']
    organization_table.loc[i, 'category'] = json_respons['data'][i]['data']['general']['category']['name']
organization_table['id'] = range(1, len(organization_table) + 1)

In [17]:
organization_table

,id,name,address,inn,description_org,category
0,1,Частный кинотеатр Black room,"обл Тверская,г Тверь,пр-кт Чайковского,д 84а",6950031262,"<p>Индивидуальные кинозалы, в которых гости см...",Кинотеатры
1,2,Горизонт Cinema&Emotion,"обл Ростовская,г Ростов-на-Дону,пр-кт Михаила ...",165000648994,<p>13-зальный мультиплекс «Горизонт. Cinema&am...,Кинотеатры
2,3,ФГБУК «Государственный фонд кинофильмов Россий...,"обл Московская,г Домодедово,мкр Белые Столбы,п...",5009007137,<p> Свою деятельность кинотеатр начал 27 авгус...,Кинотеатры
3,4,Кинотеатр «Пролетарий»,"обл Воронежская,г Воронеж,пр-кт Революции,д 56",3666112792,<p>«Пролетарий» – знаменитый кинотеатр Воронеж...,Кинотеатры
4,5,Красноярский еженедельный киноклуб,"край Красноярский,г Красноярск,ул Марковского,...",NaN,<p>КЕК – это открытый независимый киноклуб в К...,Кинотеатры
...,...,...,...,...,...,...
95,96,Сеть кинотеатров «КАРО»,"г Москва,г Москва,ул Новый Арбат,д 24",7702552238,<p>Кинотеатр «КАРО 7 Атмосфера» включает в себ...,Кинотеатры
96,97,Сеть кинотеатров «КАРО»,"г Москва,г Москва,ул Новый Арбат,д 24",7702552238,<p>Кинотеатр «КАРО 7 Калининград Плаза» работа...,Кинотеатры
97,98,Сеть кинотеатров «КАРО»,"г Москва,г Москва,ул Новый Арбат,д 24",7702552238,<p>Кинотеатр «КАРО 10 Радуга Парк» включает в ...,Кинотеатры
98,99,Сеть кинотеатров «КАРО»,"г Москва,г Москва,ул Новый Арбат,д 24",7702552238,<p>Многозальный кинотеатр «КАРО 4 Подольск» ра...,Кинотеатры


создание таблицы type_image

In [18]:
type_image_table = pd.DataFrame()
type_image_table['type_id'] = image_table['id']
for i in range(len(image_table)):
    file_name = image_table.loc[i, 'path']
    image = Image.open(file_name)
    width, height = image.size
    type_image_table.loc[i, 'type'] = file_name.split('.')[-1]
    type_image_table.loc[i, 'width'] = width
    type_image_table.loc[i, 'height'] = height
    type_image_table.loc[i, 'size'] = os.path.getsize(file_name)
type_image_table['id'] = range(1, len(type_image_table) + 1)

In [19]:
type_image_table = type_image_table[['type_id', 'id', 'type', 'size', 'width', 'height']]
type_image_table

,type_id,id,type,size,width,height
0,1,1,jpg,77089.0,1024.0,682.0
1,2,2,jpg,84246.0,1024.0,682.0
2,3,3,jpg,82857.0,1024.0,683.0
3,4,4,jpg,106233.0,1024.0,682.0
4,5,5,jpg,71293.0,1024.0,683.0
...,...,...,...,...,...,...
250,251,251,jpg,94893.0,1000.0,563.0
251,252,252,jpg,679104.0,3465.0,2599.0
252,253,253,jpg,606168.0,3283.0,2188.0
253,254,254,jpg,123338.0,1200.0,756.0


In [63]:

# сохранение таблиц
type_image_table.to_csv('type_image_table.csv', index=False)
organization_table.to_csv('organization_table.csv', index=False)
event_image_table.to_csv('event_image_table.csv', index=False)
image_table.to_csv('image_table.csv', index=False)

In [2]:
# загрузка таблиц
type_image_table = pd.read_csv('type_image_table.csv')
organization_table = pd.read_csv('organization_table.csv')
event_image_table = pd.read_csv('event_image_table.csv')
image_table = pd.read_csv('image_table.csv')

In [3]:
image_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            255 non-null    int64 
 1   url           255 non-null    object
 2   path          255 non-null    object
 3   name          255 non-null    object
 4   organization  255 non-null    object
 5   description   255 non-null    object
dtypes: int64(1), object(5)
memory usage: 12.1+ KB


In [4]:
event_image_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   id_image  255 non-null    int64
 1   id        255 non-null    int64
 2   likes     255 non-null    int64
 3   shows     255 non-null    int64
dtypes: int64(4)
memory usage: 8.1 KB


In [5]:
organization_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        100 non-null    int64  
 1   name_organization         100 non-null    object 
 2   address                   100 non-null    object 
 3   inn                       88 non-null     float64
 4   description_organization  100 non-null    object 
 5   category                  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [6]:
type_image_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   type_id  255 non-null    int64  
 1   id       255 non-null    int64  
 2   type     255 non-null    object 
 3   size     255 non-null    float64
 4   width    255 non-null    float64
 5   height   255 non-null    float64
dtypes: float64(3), int64(2), object(1)
memory usage: 12.1+ KB


In [13]:
# установка соединения с БД

conn1 = psycopg2.connect(
    database="photo4",
  user='postgres', 
  password='psswrd', 
  host='localhost', 
  port= '5432'
)
  
cursor = conn1.cursor()

In [19]:
# создание таблиц в БД

sql = '''CREATE TABLE organization(id SERIAL PRIMARY KEY NOT NULL,
                             name_organization text NOT NULL UNIQUE, 
                             address text, 
                             inn real,
                             description_organization text,
                             category text);'''

cursor.execute(sql)

sql = '''CREATE TABLE image(id SERIAL PRIMARY KEY,
                             url text, 
                             path text NOT NULL, 
                             name text NOT NULL, 
                             organization_name text NOT NULL REFERENCES organization(name_organization), 
                             description_image text);'''
  
cursor.execute(sql)
  
sql = '''CREATE TABLE event_image(id SERIAL PRIMARY KEY NOT NULL,
                             id_image integer NOT NULL REFERENCES image (id), 
                             likes numeric, 
                             shows numeric);'''

cursor.execute(sql)



sql = '''CREATE TABLE type_image(id SERIAL PRIMARY KEY NOT NULL,
                             type_id integer NOT NULL REFERENCES image (id), 
                             type varchar(10), 
                             size real, 
                             width real, 
                             height real);'''

cursor.execute(sql)

# создание индексов

sql = 'create index image_id on image(id)'
cursor.execute(sql)

sql = 'create index image_path on image(path)'
cursor.execute(sql)

sql = 'create index image_name on image(name)'
cursor.execute(sql)

sql = 'create index image_organization_name on image(organization_name)'
cursor.execute(sql)

sql = 'create index organization_category on organization USING HASH (category)'
cursor.execute(sql)

заполнение таблиц данными

In [11]:
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
  
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [ ]:
organization_table.columns = ['id', 'name_organization', 'address', 'inn', 'description_organization', 'category']

для столбца name_organization должно выполняться условие уникальности, а данные с сайта минкультуры не отвечают этому требованию, внесем изменения в таблицу

In [35]:
organization_table1 = organization_table.drop_duplicates (subset=['name_organization'])

In [39]:
organization_table1

,id,name_organization,address,inn,description_organization,category
0,1,Частный кинотеатр Black room,"обл Тверская,г Тверь,пр-кт Чайковского,д 84а",6.950031e+09,"<p>Индивидуальные кинозалы, в которых гости см...",Кинотеатры
1,2,Горизонт Cinema&Emotion,"обл Ростовская,г Ростов-на-Дону,пр-кт Михаила ...",1.650006e+11,<p>13-зальный мультиплекс «Горизонт. Cinema&am...,Кинотеатры
2,3,ФГБУК «Государственный фонд кинофильмов Россий...,"обл Московская,г Домодедово,мкр Белые Столбы,п...",5.009007e+09,<p> Свою деятельность кинотеатр начал 27 авгус...,Кинотеатры
3,4,Кинотеатр «Пролетарий»,"обл Воронежская,г Воронеж,пр-кт Революции,д 56",3.666113e+09,<p>«Пролетарий» – знаменитый кинотеатр Воронеж...,Кинотеатры
4,5,Красноярский еженедельный киноклуб,"край Красноярский,г Красноярск,ул Марковского,...",NaN,<p>КЕК – это открытый независимый киноклуб в К...,Кинотеатры
...,...,...,...,...,...,...
72,73,МБУК «Белевский центр развития культуры и тури...,"обл Тульская,р-н Белевский,г Белев,ул Советска...",7.122001e+09,<p>1 ноября 2016 года на базе Белевского центр...,Кинотеатры
73,74,ООО «Наше кино»,"обл Архангельская,г Северодвинск,ул Карла Марк...",2.902065e+09,<p>Кинотеатр был открыт в 1970 году. С 1993 го...,Кинотеатры
74,75,Сеть кинотеатров «КАРО»,"г Москва,г Москва,ул Новый Арбат,д 24",7.702552e+09,<p>Сеть «КАРО» открыла первый кинотеатр нового...,Кинотеатры
78,79,ООО «КРК «Звезда»,"обл Тверская,г Тверь,наб Степана Разина,д 1",6.950122e+09,<p>Кинотеатр «Звезда» был построен в 1937 году...,Кинотеатры


In [37]:
execute_values(conn1, organization_table1, 'organization')

execute_values() done


In [65]:
image_table.columns = ['id', 'url', 'path', 'name', 'organization_name', 'description_image']
execute_values(conn1, image_table, 'image')

execute_values() done


In [48]:
event_image_table.columns = ['id', 'id_image', 'likes', 'shows']
copy_event_image_table = pd.DataFrame()
for i in range(len(event_image_table)):
    copy_event_image_table.loc[i, 'id'] = event_image_table.loc[i, 'id'].item()
    copy_event_image_table.loc[i, 'id_image'] = event_image_table.loc[i, 'id_image'].item()
    copy_event_image_table.loc[i, 'likes'] = event_image_table.loc[i, 'likes'].item()
    copy_event_image_table.loc[i, 'shows'] = event_image_table.loc[i, 'shows'].item()

In [52]:
execute_values(conn1, copy_event_image_table, 'event_image')

execute_values() done


In [53]:
type_image_table.columns = ['id', 'type_id', 'type', 'size', 'width', 'height']
execute_values(conn1, type_image_table, 'type_image')

execute_values() done


In [54]:
conn1.commit()
conn1.close()